In [1]:
import os
import sys

sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"
os.environ["NUMBA_DEBUG_CACHE"] = "1"


In [2]:
from numba.experimental import jitclass
from numba import njit
import numpy as np
from my_stuff import MufexKeys

from nb_quantfreedom.exchanges.mufex_exchange.mufex import Mufex
mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2


In [3]:
candles = mufex_main.get_candles_df(
    symbol='BTCUSDT',
    timeframe="1m",
    since_date_ms=1697237940000,
    until_date_ms=1697241540000,
)

It took 00 mins and 00 seconds to download 61 candles


In [4]:
@njit(cache=True)
def sl_price_getter(
    candle_column: int,
    lookback: int,
    bar_index: int,
    candles: np.array,
):
    price = candles[lookback : bar_index + 1, candle_column].min()
    return price

In [5]:
class Leverage:
    def __init__(self) -> None:
        pass

    def market_fee_pct(self):
        return 0.0009

    def mmr_pct(self):
        return 0.005

    def max_leverage(self):
        return 150.0

    def leverage_calculator(self):
        raise Exception()

    def calc_liq_price(
        self,
        leverage: float,
        entry_size_usd: float,
        average_entry: float,
        og_available_balance: float,
        og_cash_used: float,
        og_cash_borrowed: float,
    ):
        # Getting Order Cost
        # https://www.bybithelp.com/HelpCenterKnowledge/bybitHC_Article?id=000001064&language=en_US
        initial_margin = entry_size_usd / leverage
        fee_to_open = entry_size_usd * 0.0009  # math checked
        possible_bankruptcy_fee = entry_size_usd * (leverage - 1) / leverage * self.mmr_pct()
        cash_used = initial_margin + fee_to_open + possible_bankruptcy_fee  # math checked

        if cash_used > og_available_balance:
            raise Exception(
                msg=f"Cash used={cash_used} > available_balance={og_available_balance}",
                order_status=1,
            )
        else:
            # liq formula
            # https://www.bybithelp.com/HelpCenterKnowledge/bybitHC_Article?id=000001067&language=en_US
            available_balance = round(og_available_balance - cash_used, 4)
            cash_used = round(og_cash_used + cash_used, 4)
            cash_borrowed = round(og_cash_borrowed + entry_size_usd - cash_used, 4)

            liq_price = average_entry * (1 - (1 / leverage) + self.mmr_pct())  # math checked
            liq_price = liq_price
            can_move_sl_to_be = True

        return (
            leverage,
            liq_price,
            available_balance,
            cash_used,
            cash_borrowed,
            can_move_sl_to_be,
        )


@jitclass()
class SetStaticLeverage(Leverage):
    def leverage_calculator(
        self,
        static_leverage: float,
        average_entry: float,
        entry_size_usd: float,
        cash_used: float,
        available_balance: float,
        cash_borrowed: float,
        sl_price: float,
    ):
        liq_price = self.calc_liq_price(
            leverage=static_leverage,
            entry_size_usd=entry_size_usd,
            average_entry=average_entry,
            og_cash_used=cash_used,
            og_available_balance=available_balance,
            og_cash_borrowed=cash_borrowed,
        )
        return liq_price


@jitclass()
class CalcDynamicLeverage(Leverage):
    def leverage_calculator(
        self,
        static_leverage: float,
        average_entry: float,
        entry_size_usd: float,
        cash_used: float,
        available_balance: float,
        cash_borrowed: float,
        sl_price: float,
    ):
        leverage = -average_entry / ((sl_price - sl_price * 0.001) - average_entry - self.mmr_pct() * average_entry)

        if leverage > self.max_leverage():
            # print(f"Setting leverage from {leverage} to max leverage {self.max_leverage()}")
            leverage = self.max_leverage()
        elif leverage < 1:
            # print(f"Setting leverage from {leverage} to {1}")
            leverage = 1
        else:
            # print(f"Leverage set too {leverage}")
            leverage = leverage

        liq_price = self.calc_liq_price(
            leverage=leverage,
            entry_size_usd=entry_size_usd,
            average_entry=average_entry,
            og_cash_used=cash_used,
            og_available_balance=available_balance,
            og_cash_borrowed=cash_borrowed,
        )
        return liq_price

In [16]:
@njit(cache=True)
def apply(num):
    lev_calc = (SetStaticLeverage(), CalcDynamicLeverage())
    leverage_calculator = lev_calc[num]
    leverage_resuls = leverage_calculator.leverage_calculator(
        static_leverage=20.0,
        average_entry=26557.6,
        entry_size_usd=26.5513,
        cash_used=10.0,
        available_balance=6948.548,
        cash_borrowed=643.15,
        sl_price=26544.2,
    )
    return leverage_resuls



In [17]:
leverage_calculator = (SetStaticLeverage(), CalcDynamicLeverage())

# print(apply(leverage_calculator[0]))
apply(1)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mNo implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(Tuple(instance.jitclass.SetStaticLeverage#21283d9bfa0<>, instance.jitclass.CalcDynamicLeverage#212b4c6fac0<>), int64)
 
There are 22 candidate implementations:
[1m  - Of which 22 did not match due to:
  Overload of function 'getitem': File: <numerous>: Line N/A.
    With argument(s): '(Tuple(instance.jitclass.SetStaticLeverage#21283d9bfa0<>, instance.jitclass.CalcDynamicLeverage#212b4c6fac0<>), int64)':[0m
[1m   No match.[0m
[0m
[0m[1mDuring: typing of intrinsic-call at C:\Users\User\AppData\Local\Temp\ipykernel_35024\800625905.py (4)[0m
[1m
File "C:\Users\User\AppData\Local\Temp\ipykernel_35024\800625905.py", line 4:[0m
[1mdef apply(num):
    <source elided>
    lev_calc = (SetStaticLeverage(), CalcDynamicLeverage())
[1m    leverage_calculator = lev_calc[num]
[0m    [1m^[0m[0m
